In [4]:
#import libraries
import numpy as np
import pandas as pd
import gzip
import fastparquet

In [5]:
#read feature space file to get target objects
features = pd.read_csv("metamdp_feature_space.csv")
targets = np.unique(features['object'])

#use all objects but 'Pad'
print(targets)

['Airplane' 'Allergy Pills' 'Apple' 'Bacon' 'Banana' 'Beans' 'Block 1'
 'Block 10' 'Block 2' 'Block 3' 'Block 4' 'Block 5' 'Block 6' 'Block 7'
 'Block 8' 'Block 9' 'Book 1' 'Book 10' 'Book 11' 'Book 12' 'Book 2'
 'Book 3' 'Book 4' 'Book 5' 'Book 6' 'Book 7' 'Book 8' 'Book 9' 'Bread'
 'Candle' 'Candlestick' 'Carrot' 'Clipboard' 'Coconut' 'Coffee' 'Crayon 1'
 'Crayon 2' 'Crayon 3' 'Crayon 4' 'Crayon 5' 'Crayon 6' 'Cup' 'Cylinder 1'
 'Cylinder 2' 'Cylinder 3' 'Cylinder 4' 'Dodecahedron' 'Donut 1' 'Donut 2'
 'Eraser' 'Gold' 'Helmet' 'Juice Box' 'Key' 'Knight' 'Laptop' 'Left Shoe'
 'Magazine 1' 'Magazine 2' 'Marker 1' 'Marker 2' 'Marker 3' 'Marker 4'
 'Milk Carton 1' 'Milk Carton 2' 'Milk Carton 3' 'Mug' 'Notebook'
 'Oil Bottle 1' 'Oil Bottle 2' 'Oil Carafe 1' 'Oil Carafe 2' 'Pad'
 'Paint Can' 'Paper' 'Paper Bag' 'Paper Box' 'Pear' 'Pen' 'Pencil 1'
 'Pencil 2' 'Phone' 'Picture 1' 'Picture 2' 'Pills' 'Pizza Box'
 'Plastic Box 1' 'Plastic Box 2' 'Raw meat' 'Right Shoe' 'Rubber Duck'
 'Scissor

In [6]:
#unzip conceptnet dataset (this takes a while!)
with gzip.open('conceptnet-assertions-5.7.0.csv.gz') as f:
    conceptnet = pd.read_csv(f,sep='\t',names=['uri','relation','start_node','end_node','info'])

In [8]:
#refer to concept net edge types to choose a relation: https://github.com/commonsense/conceptnet5/wiki/Relations
relation = '/r/AtLocation'

#choose the language of indexed concepts
language = '/c/en'

#keep only the data associated with chosen relation nad language combo
conceptnet_new = conceptnet[conceptnet['relation'] == relation]
conceptnet_new = conceptnet_new[conceptnet_new['start_node'].str.contains(language) & conceptnet_new['end_node'].str.contains(language)]


In [21]:
#choose object for which to search relations
object =  'fruit'
object = '/c/en/' + object

#choose whether the object will be the start or end node relation
node = 'start_node'

#get relations for specified objects and associated info for each relationship (make more specific to object)
object_relations = conceptnet_new[conceptnet_new[node] == object]
info = object_relations['info']


In [22]:
#extract weight for each relation 
weights = []

for i in range(info.shape[0]):
    x = info.iloc[i].split(":")
    y = x[len(x)-1][1:-1]
    weights.append(y)
    
weights = np.asarray(weights)
weights


array(['1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '4.0', '1.0', '1.0',
       '1.0', '1.0', '1.0', '2.0', '1.0', '1.0', '4.0'], dtype='<U3')

In [23]:
#sort relations according to weights (from highest to lowest weight)
idx = np.asarray(np.flip(np.argsort(weights)))

#print start and end nodes in order from strongest to weakest relationship
object_relations.iloc[:,-3:-1].iloc[idx]

,start_node,end_node
75979,/c/en/fruit,/c/en/tree
75970,/c/en/fruit,/c/en/market
75976,/c/en/fruit,/c/en/refridgerator
75978,/c/en/fruit,/c/en/san_francisco
75977,/c/en/fruit,/c/en/refrigerator
75975,/c/en/fruit,/c/en/refriderator
75974,/c/en/fruit,/c/en/produce_sections
75973,/c/en/fruit,/c/en/plate
75972,/c/en/fruit,/c/en/pantry
75971,/c/en/fruit,/c/en/mouth
